# Assignment 2: IR

## Preparations
* Put all your imports, and path constants in the next cells
* Make sure all your path constants are **relative to** ***DATA_DIR*** and **NOT hard-coded** in your code.

In [0]:
!pip install whoosh
!pip install pytrec_eval
!pip install wget

In [0]:
import wget
wget.download("https://github.com/MIE451-1513-2019/course-datasets/raw/master/government.zip", "government.zip")

In [0]:
!unzip government.zip

In [0]:
# imports
# Put all your imports here

In [0]:
DATA_DIR = "government"
#
# Put other path constants here
#

## Question 1
Provide your text answers in the following two markdown cells

### Q1 (a): Provide answer to Q1 (a) here [markdown cell]

### Q1 (b): Provide answer to Q1 (b) here [markdown cell]

## Question 2

### Q2 (a): Write your code below

In [0]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q2, your query parser in QP_Q2, and your searcher in SEARCHER_Q2

In [0]:
INDEX_Q2 = None # Replace None with your index for Q2
QP_Q2 = None # Replace None with your query parser for Q2
SEARCHER_Q2 = None # Replace None with your searcher for Q2

### Q2 (b): Provide answer to Q2 (b) here [markdown cell]

### Q2 (c): Provide answer to Q2(c) here [markdown cell]

## Question 3

### Q3 (a): Provide answer to Q3 (a) here [markdown cell]

### Q3 (b): Write your code below

In [0]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q3, your query parser in QP_Q3, and your searcher in SEARCHER_Q3

In [0]:
INDEX_Q3 = None # Replace None with your index for Q3
QP_Q3 = None # Replace None with your query parser for Q3
SEARCHER_Q3 = None # Replace None with your searcher for Q3

### Q3 (c): Provide answer to Q3 (c) here [markdown cell]

### Q3 (d): Provide answer to Q3 (d) here [markdown cell]

### Q3 (e): Provide answer to Q3 (e) here [markdown cell]

### Q3 (f): Provide answer to Q3 (f) here [markdown cell]

## Question 4

In [0]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q4, your query parser in QP_Q4, and your searcher in SEARCHER_Q4

### Please answer the following questions here
(a) A clear list of all final modifications made.  
(b)  Why each modification was made – how did it help?  
(c)  The  final  MAP  performance  that  these  modifications  attained.

In [0]:
INDEX_Q4 = None # Replace None with your index for Q4
QP_Q4 = None # Replace None with your query parser for Q4
SEARCHER_Q4 = None # Replace None with your searcher for Q4

## Validation

In [0]:
# Run the following cells to make sure your code returns the correct value types

In [0]:
from whoosh.index import FileIndex
from whoosh.qparser import QueryParser
from whoosh.searching import Searcher
import os.path

### Q2 Validation

In [0]:
assert(isinstance(INDEX_Q2, FileIndex)), "Index Type"
assert(isinstance(QP_Q2, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q2, Searcher)), "Searcher Type"
print("Q2 Types Validated")

### Q3 Validation

In [0]:
assert(isinstance(INDEX_Q3, FileIndex)), "Index Type"
assert(isinstance(QP_Q3, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q3, Searcher)), "Searcher Type"
print("Q3 Types Validated")

### Q4 Validation

In [0]:
assert(isinstance(INDEX_Q4, FileIndex)), "Index Type"
assert(isinstance(QP_Q4, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q4, Searcher)), "Searcher Type"
print("Q4 Types Validated")